In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - LLaMA 2 (Evaluation)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_llama2_evaluation.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_evaluation.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates downloading prebuilt [LLaMA 2 models](https://huggingface.co/meta-llama), evaluating LLaMA 2 models with popular benchmark datasets through Vertex CustomJobs using [EleutherAI's evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness) and running
[automatic side-by-side evaluation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/side-by-side-eval).

### Objective

- Download prebuilt LLaMA 2 models
- Evaluate the LLaMA 2 models on any of the benchmark datasets
- Run bulk inference job
- Run automatic side by side (autoSxS) evaluation job

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing
# @markdown experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`)
# @markdown should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is
# @markdown not considered a match for a single region covered by the multi-region range (eg. "us-central1").
# @markdown If not set, a unique GCS bucket will be created instead.

# Import the necessary packages
! pip3 install --upgrade --quiet google-cloud-aiplatform google-cloud-pipeline-components

import importlib
import json
import os
import uuid
from datetime import datetime
from typing import Dict

import pandas as pd
from google.cloud import aiplatform, storage

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type:"string"}

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "llama2")
BASE_MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "base_model")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")
OUTPUT_BUCKET_A = os.path.join(EXPERIMENT_BUCKET, "output_a")
OUTPUT_BUCKET_B = os.path.join(EXPERIMENT_BUCKET, "output_b")

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID

# The evaluation and the bulk inference docker images.
EVAL_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-lm-evaluation-harness:20231011_0934_RC00"
BULK_INFERRER_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-bulk-inferrer:20240708_1042_RC00"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering jobs in Vertex AI."""
    return prefix + datetime.now().strftime("-%Y%m%d%H%M%S")


def preprocess(
    output_prediction_a: Dict[str, str], output_prediction_b: Dict[str, str]
) -> Dict[str, str]:
    """Preprocesses the output predictions of model a and model b.

    It takes the output predictions of bulk inference job of the model a and
    model b and merges into one jsonl file.

    Args:
      output_prediction_a:
        Output json file which contains the predictions of the model a.
      output_prediction_b:
        Output json file which contains the predictions of the model b.

    Returns:
      Merged jsonl file.
    """
    # Get the outputs of prediction of to the dataframe.
    df1 = pd.read_json(output_prediction_a, lines=True)
    df2 = pd.read_json(output_prediction_b, lines=True)

    # Rename the columns and merge the dataframes based on the input column.
    df1 = df1.rename(columns={index_column: "inputs", "prediction": "pred_a"})
    df2 = df2.rename(columns={index_column: "inputs", "prediction": "pred_b"})

    df1["inputs"] = df1["inputs"].apply(lambda d: d["inputs_pretokenized"])
    df2["inputs"] = df2["inputs"].apply(lambda d: d["inputs_pretokenized"])

    result = pd.merge(df1, df2, on=index_column)

    # Convert the dataframe to result.jsonl file.
    return result.to_json("result.jsonl", orient="records", lines=True)

In [ ]:
# @title Access pretrained LLaMA 2 models

# @markdown The original models from Meta are converted into the HuggingFace format for serving in Vertex AI.

# @markdown Accept the model agreement to access the models:
# @markdown 1. Open the [LLaMA 2 model card](https://console.cloud.google.com/vertex-ai/publishers/meta/model-garden/llama2).
# @markdown 2. Review and accept the agreement in the pop-up window on the model card page. If you have previously accepted the model agreement, there will not be a pop-up window on the model card page and this step is not needed.
# @markdown 3. A Cloud Storage bucket (starting with ‘gs://’) containing LLaMA 2 pretrained and finetuned models will be shared under the “Documentation” section and its “Get started” subsection.

# This path will be shared once click the agreement in Code LLaMA model card
# as described in the `Access pretrained Code LLaMA models` section.
VERTEX_AI_MODEL_GARDEN_LLAMA2 = ""  # @param {type:"string"}
assert (
    VERTEX_AI_MODEL_GARDEN_LLAMA2
), "Click the agreement of LLaMA2 in Vertex AI Model Garden, and get the GCS path of LLaMA2 model artifacts."
print(
    "Copy LLaMA2 model artifacts from",
    VERTEX_AI_MODEL_GARDEN_LLAMA2,
    "to ",
    BASE_MODEL_BUCKET,
)
! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_LLAMA2/* $BASE_MODEL_BUCKET

In [ ]:
# @title Evaluate LLaMA 2 models
# @markdown This section demonstrates evaluation of LLaMA 2 models using EleutherAI's [Language Model Evaluation Harness (lm-evaluation-harness)](https://github.com/EleutherAI/lm-evaluation-harness) with Vertex Custom Job.

# @markdown This example uses the dataset [TruthfulQA](https://arxiv.org/abs/2109.07958).
# @markdown All the supported tasks are listed in [this task table](https://github.com/EleutherAI/lm-evaluation-harness/blob/master/docs/task_table.md).

# set the base model id
base_model_name = "llama2-7b-chat-hf"  # @param ["llama2-7b-hf", "llama2-7b-chat-hf", "llama2-13b-hf", "llama2-13b-chat-hf", "llama2-70b-hf", "llama2-70b-chat-hf"]
base_model_id = os.path.join(BASE_MODEL_BUCKET, base_model_name)

# Set the machine_type, accelerator_type, accelerator_count and benchmark dataset.
eval_dataset = "truthfulqa_mc"  # @param ["truthfulqa_mc", "boolq", "gsm8k", "hellaswag", "natural_questions", "openai_humaneval", "openbookqa", "quac", "trivia_qa", "winograde"]

# Worker pool spec.
# Find Vertex AI supported accelerators and regions in:
#  https://cloud.google.com/vertex-ai/docs/training/configure-compute

if base_model_name == "llama2-7b-hf":
    # Sets 1 (24G) to evaluate LLaMA2 7B models.
    machine_type = "g2-standard-16"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 1
elif base_model_name == "llama2-7b-chat-hf":
    # Sets 1 L4 (24G) to evaluate LLaMA2 7B models.
    machine_type = "g2-standard-16"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 1
elif base_model_name == "llama2-13b-hf":
    # Sets 2 L4 (24G) to evaluate LLaMA2 13B models.
    machine_type = "g2-standard-24"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 2
elif base_model_name == "llama2-13b-chat-hf":
    # Sets 2 L4 (24G) to evaluate LLaMA2 13B models.
    machine_type = "g2-standard-24"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 2
elif base_model_name == "llama2-70b-hf":
    # Sets 8 L4 (24G) to evaluate LLaMA2 70B models.
    machine_type = "g2-standard-96"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 8
elif base_model_name == "llama2-70b-chat-hf":
    # Sets 8 L4 (24G) to evaluate LLaMA2 70B models.
    machine_type = "g2-standard-96"
    accelerator_type = "NVIDIA_L4"
    accelerator_count = 8

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=True,
)

replica_count = 1

job_name = get_job_name_with_datetime(prefix="llama2-eval")
eval_output_dir = os.path.join(MODEL_BUCKET, job_name)
eval_output_dir_gcsfuse = eval_output_dir.replace("gs://", "/gcs/")
base_model_id_gcsfuse = base_model_id.replace("gs://", "/gcs/")

# @markdown  To evaluate a PEFT-finetuned model, enter the PEFT output directory below.
# @markdown  Otherwise, leave it empty.

# @markdown  See the [finetuning notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_peft_finetuning.ipynb) for more details:

peft_output_dir = ""  # @param {type:"string"}
peft_output_dir_gcsfuse = peft_output_dir.replace("gs://", "/gcs/")

# Prepare evaluation command that runs the evaluation harness.
# Set `trust_remote_code = True` because evaluating the model requires
# executing code from the model repository.
# Set `use_accelerate = True` to enable evaluation across multiple GPUs.
eval_command = [
    "python",
    "main.py",
    "--model",
    "hf-causal-experimental",
    "--tasks",
    f"{eval_dataset}",
    "--output_path",
    f"{eval_output_dir_gcsfuse}",
]

if peft_output_dir_gcsfuse:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id_gcsfuse},peft={peft_output_dir_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    ]
else:
    eval_command += [
        "--model_args",
        f"pretrained={base_model_id_gcsfuse},trust_remote_code=True,use_accelerate=True,device_map_option=auto",
    ]

# Run the evaluation job.
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": EVAL_DOCKER_URI,
            "command": eval_command,
            "args": [],
        },
    }
]

eval_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=eval_output_dir,
)

eval_job.run()

print("Evaluation results were saved in:", eval_output_dir)

# Fetch evaluation results.
storage_client = storage.Client()
BUCKET_NAME = BUCKET_URI.replace("gs://", "")
bucket = storage_client.get_bucket(BUCKET_NAME)
RESULT_FILE_PATH = eval_output_dir[len(BUCKET_URI) + 1 :]
blob = bucket.blob(RESULT_FILE_PATH)
raw_result = blob.download_as_string()

# Print evaluation results.
result = json.loads(raw_result)
result_formatted = json.dumps(result, indent=2)
print(f"Evaluation result:\n{result_formatted}")

### Bulk Inference

In [ ]:
# @title [Optional] Generate `input_dataset` for the bulk inference job
# @markdown Note: For experimentation, we request that users provide only a few prompts.

# @markdown For demonstration, a publicly available [TensorFlow dataset](https://www.tensorflow.org/datasets/catalog/reddit) is used. This dataset contains preprocessed posts from the Reddit dataset.
TEST_DATASET = "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/val/shard-00000-of-00001.jsonl"  # @param {type:"string"}
NUM_EXAMPLES = 50  # @param {type:"integer"}

# Load dataset and modify.
df = pd.read_json(TEST_DATASET, lines=True)
examples = df.head(NUM_EXAMPLES)

# Upload new dataset to GCS.
examples.to_json("data.json", orient="records", lines=True)
! gsutil cp data.json $BUCKET_URI/temp/data.jsonl
DATASET = f"{BUCKET_URI}/temp/data.jsonl"

print(f"{NUM_EXAMPLES} examples written to {DATASET}.")

In [ ]:
# @title Set up bulk inference job

# @markdown Run the bulk inference custom job to generate offline predictions.
# @markdown You can perform bulk inference using a base model or LoRA finetuned model.

# Setup bulk inference job.
model_a = "llama2-7b-hf"  # @param ["llama2-7b-hf", "llama2-7b-chat-hf", "llama2-13b-hf", "llama2-13b-chat-hf",  "llama2-70b-hf", "llama2-70b-chat-hf"]
model_b = "llama2-13b-hf"  # @param ["llama2-7b-hf", "llama2-7b-chat-hf", "llama2-13b-hf", "llama2-13b-chat-hf",  "llama2-70b-hf", "llama2-70b-chat-hf"]

# @markdown **Required Parameters**

# @markdown `input_dataset` : Path to JSONL file containing the input dataset.

# @markdown `index_column` : The column which distinguishes unique evaluation examples.

# @markdown `input_text` : Indexing key for inputs.

# @markdown `output_prediction_a`: Path to JSONL file which will contain output predictions of model a.

# @markdown `output_prediction_b`: Path to JSONL file which will contain output predictions of model b.

input_dataset = f"{BUCKET_URI}/temp/data.jsonl"  # @param {type:"string"}
index_column = "inputs"  # @param {type:"string"}
input_text = "input_text"  # @param {type:"string"}

output_prediction_a = ""  # @param {type:"string"}
output_prediction_b = ""  # @param {type:"string"}


# @markdown **Optional Parameters** : Provide the path to the LoRA finetuned models.

# @markdown `lora_path_a` : Path to finetuned LoRA adapter of model a.

# @markdown `lora_path_b` : Path to finetuned LoRA adapter of model b.

lora_path_a = ""  # @param {type:"string"}
lora_path_b = ""  # @param {type:"string"}


if model_a == model_b:
    raise ValueError("Select different models to run AutoSxS evaluation.")

if model_a in ["llama2-7b-hf", "llama2-7b-chat-hf"]:
    machine_type_a = "g2-standard-16"
    accelerator_type_a = "NVIDIA_L4"
    accelerator_count_a = 1
elif model_a in ["llama2-13b-hf", "llama2-13b-chat-hf"]:
    machine_type_a = "g2-standard-24"
    accelerator_type_a = "NVIDIA_L4"
    accelerator_count_a = 2
elif model_a in ["llama2-70b-hf", "llama2-70b-chat-hf"]:
    machine_type_a = "g2-standard-96"
    accelerator_type_a = "NVIDIA_L4"
    accelerator_count_a = 8

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type_a,
    accelerator_count=accelerator_count_a,
    is_for_training=True,
)

if model_b in ["llama2-7b-hf", "llama2-7b-chat-hf"]:
    machine_type_b = "g2-standard-16"
    accelerator_type_b = "NVIDIA_L4"
    accelerator_count_b = 1
elif model_b in ["llama2-13b-hf", "llama2-13b-chat-hf"]:
    machine_type_b = "g2-standard-24"
    accelerator_type_b = "NVIDIA_L4"
    accelerator_count_b = 2
elif model_b in ["llama2-70b-hf", "llama2-70b-chat-hf"]:
    machine_type_b = "g2-standard-96"
    accelerator_type_b = "NVIDIA_L4"
    accelerator_count_b = 8

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type_b,
    accelerator_count=accelerator_count_b,
    is_for_training=True,
)

replica_count = 1

bulk_infer_job_name = get_job_name_with_datetime(prefix="bulk-infer")
eval_output_dir = os.path.join(MODEL_BUCKET, bulk_infer_job_name)

# Maximum encoder/prefix length. Inputs will be padded or truncated to match this length.
input_seq_length = 50

# Maximum number of decoder steps. Outputs will be at most this length.
targets_seq_length = 50

model_id_a = os.path.join(BASE_MODEL_BUCKET, model_a)
model_id_b = os.path.join(BASE_MODEL_BUCKET, model_b)

worker_pool_specs_base_model_a = [
    {
        "machine_spec": {
            "machine_type": machine_type_a,
            "accelerator_type": accelerator_type_a,
            "accelerator_count": accelerator_count_a,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_type": "pd-ssd",
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": BULK_INFERRER_DOCKER_URI,
            "args": [
                f"--large_model_reference={model_id_a}",
                f"--input_model={lora_path_a}",
                f"--input_dataset={input_dataset}",
                "--dataset_split=empty",
                f"--output_prediction={output_prediction_a}",
                f"--output_prediction_gcs_path={OUTPUT_BUCKET_A}",
                f"--inputs_sequence_length={input_seq_length}",
                f"--targets_sequence_length={targets_seq_length}",
                f"--inputs_key={input_text}",
            ],
        },
    }
]

worker_pool_specs_base_model_b = [
    {
        "machine_spec": {
            "machine_type": machine_type_b,
            "accelerator_type": accelerator_type_b,
            "accelerator_count": accelerator_count_b,
        },
        "replica_count": replica_count,
        "disk_spec": {
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": BULK_INFERRER_DOCKER_URI,
            "args": [
                f"--large_model_reference={model_id_b}",
                f"--input_model={lora_path_b}",
                f"--input_dataset={input_dataset}",
                "--dataset_split=empty",
                f"--output_prediction={output_prediction_b}",
                f"--output_prediction_gcs_path={OUTPUT_BUCKET_B}",
                f"--inputs_sequence_length={input_seq_length}",
                f"--targets_sequence_length={targets_seq_length}",
                f"--inputs_key={input_text}",
            ],
        },
    }
]

In [ ]:
# @title Run bulk inference job for Model A

bulk_inferrer_a = aiplatform.CustomJob(
    display_name=get_job_name_with_datetime(prefix="bulk-infer-a"),
    worker_pool_specs=worker_pool_specs_base_model_a,
    base_output_dir=os.path.join(OUTPUT_BUCKET_A, bulk_infer_job_name),
)

bulk_inferrer_a.run()

In [ ]:
# @title Run bulk inference job for Model B

bulk_inferrer_b = aiplatform.CustomJob(
    display_name=get_job_name_with_datetime(prefix="bulk-infer-b"),
    worker_pool_specs=worker_pool_specs_base_model_b,
    base_output_dir=os.path.join(OUTPUT_BUCKET_B, bulk_infer_job_name),
)

bulk_inferrer_b.run()

### AutoSxS Job

In [ ]:
# @title Compile AutoSxS pipeline

from google_cloud_pipeline_components.preview import model_evaluation
from kfp import compiler

template_uri = "pipeline.yaml"
compiler.Compiler().compile(
    pipeline_func=model_evaluation.autosxs_pipeline,
    package_path=template_uri,
)

In [ ]:
# @title Run AutoSxS pipeline job

# @markdown Automatic side-by-side (AutoSxS) is a model-assisted evaluation tool that compares two large language models (LLMs) side by side.
# @markdown In order to run AutoSxS, we need to define a `autosxs_pipeline` job with the following parameters. More details of the AutoSxS pipeline configuration can be found [here](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/preview/model_evaluation.html#preview.model_evaluation.autosxs_pipeline).


# Preprocess the output json files and copy to the GCS bucket
preprocess(output_prediction_a, output_prediction_b)
PREDS = f"{BUCKET_URI}/temp/preds/"
! gsutil cp result.jsonl $PREDS

# @title AutoSxS Job
autosxs_job_name = get_job_name_with_datetime(prefix="autosxs")

# @markdown AutoSxS supports evaluating models for summarization and question-answering tasks.
"""
Evaluation task in the form {task}@{version}. Task can be one of
[summarization, question_answer].
version is an integer with three digits or 'latest'.
Ex: summarization@001 or question_answer@latest
"""
task_name = "summarization@001"  # @param ["question_answer@latest", "summarization@001"]


parameters = {
    "evaluation_dataset": f"{BUCKET_URI}/temp/preds/result.jsonl",
    "id_columns": ["inputs"],
    "autorater_prompt_parameters": {
        "inference_context": {"column": index_column},
        "inference_instruction": {"template": "{{ default_instruction }}"},
    },
    "response_column_a": "pred_a",
    "response_column_b": "pred_b",
    "task": task_name,
}

autosxs_job = aiplatform.PipelineJob(
    job_id=autosxs_job_name,
    display_name=autosxs_job_name,
    pipeline_root=os.path.join(BUCKET_URI, autosxs_job_name),
    template_path=template_uri,
    parameter_values=parameters,
    enable_caching=False,
)
autosxs_job.run()

In [ ]:
# @title Get autorater judgements
# @markdown Autorater is a language model which compares the quality of two model responses based on a pre-defined criteria.
# @markdown More details on the autorater can be found [here](https://cloud.google.com/vertex-ai/generative-ai/docs/models/side-by-side-eval#autorater)

for details in autosxs_job.task_details:
    if details.task_name == "online-evaluation-pairwise":
        break

# Judgments
judgments_uri = details.outputs["judgments"].artifacts[0].uri
judgments_df = pd.read_json(judgments_uri, lines=True)
judgments_df.head()

In [ ]:
# @title Get win-rate
# @markdown Win rate is the percentage of the time the autorater has decided that a particular model had a better response.

for details in autosxs_job.task_details:
    if details.task_name == "model-evaluation-text-generation-pairwise":
        break
pd.DataFrame([details.outputs["autosxs_metrics"].artifacts[0].metadata])

### Clean up resources

In [ ]:
# @title Clean up
# @markdown  Delete the jobs to recycle the resources and avoid unnecessary continouous charges that may incur.

eval_job.delete()
bulk_inferrer_a.delete()
bulk_inferrer_b.delete()
autosxs_job.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False  # @param {type: "boolean"}
if delete_bucket:
    ! gsutil -m rm -r $EXPERIMENT_BUCKET